# Cleaning data for models and visualization

First steps in cleaning data are to make all the values that can be entirly numeric

Before we do that we must import our utility scripts

In [1]:
import sys
import os
import pandas as pd
import numpy as  np

project_dir = '/home/atoris/course-project-thomas-wright/src'
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)

In [2]:
import datautil as du

But first we must be able to load our data

In [3]:
url = 'https://www.cpubenchmark.net/mid_range_cpus.html'
filename = 'cpu_data.csv'

df = du.load_data(url, filename)
df.head(5)

/usr/home/atoris/course-project-thomas-wright/data/cpu_data.csv


,name,price,Class,Socket,Clockspeed,Turbo Speed,Threads,Cores,Typical TDP,mt_score,st_score
0,AMD Ryzen Threadripper PRO 3995WX,NaN,Desktop,sTRX4,2.7,4.2,128,64,280W,88731,2665
1,AMD Ryzen Threadripper 3990X,"$4,999.99",Desktop,sTRX4,2.9,4.3,128,64,280W,80558,2556
2,AMD EPYC 7702,"$4,955.00",Server,SP3,2.0,3.4,128,64,200W,71859,2096
3,AMD EPYC 7702P,"$4,749.95",Server,SP3,2.0,3.4,128,64,200W,68213,2210
4,AMD EPYC 7742,"$6,790.00",Server,SP3,2.3,3.4,128,64,225W,67185,2376


It looks like we could remove the units from price and TDP and encode our Class with a OneHot encode,
I will start with the units

In [4]:
for index, value in df.iterrows():
    price = str(value['price']).replace("*", "")
    TDP = str(value['Typical TDP'])
    
    if price[0] == "$":
        price = price[1:]
        price = price.replace(",", "")
    if TDP[-1:] == "W":
        TDP = TDP[:-1]
    if ((TDP[-1:] == "3") or (TDP[-1:] == "2")):
        if(TDP[-2:-1] == "W"):
            TDP = TDP[:-2]
        
    
    
    df.at[index, 'price'] = price
    df.at[index, 'Typical TDP'] = TDP


In [5]:
df.head(1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         740 non-null    object 
 1   price        740 non-null    object 
 2   Class        712 non-null    object 
 3   Socket       706 non-null    object 
 4   Clockspeed   740 non-null    float64
 5   Turbo Speed  436 non-null    float64
 6   Threads      740 non-null    int64  
 7   Cores        740 non-null    int64  
 8   Typical TDP  740 non-null    object 
 9   mt_score     740 non-null    int64  
 10  st_score     740 non-null    int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 63.7+ KB


In this dataset singlethreaded_score should always be less than mulithreaded_score so we should verify that this is correct. Some errors happen during data collection process that could make this false so we should fix them by dropping the row or getting the correct value.

In [6]:
for index, value in df.iterrows():
    st_score = value['st_score']
    mt_score = value['mt_score']
    
    if st_score > mt_score:
        st_score = str(st_score)[:-1]
    
    df.at[index, 'st_score'] = int(st_score)
df.head(1)

,name,price,Class,Socket,Clockspeed,Turbo Speed,Threads,Cores,Typical TDP,mt_score,st_score
0,AMD Ryzen Threadripper PRO 3995WX,nan,Desktop,sTRX4,2.7,4.2,128,64,280,88731,2665


This data looks good except the names not look a bit messy and contain redudant data so we will split the data up into Brand and name and remove the clock speed as it's data already has a column

In [7]:
for index, value in df.iterrows():
    name = str(value['name'])
    
    if name[0] == 'I':
        df.at[index, 'brand'] = 'Intel'
        name = name[6:]
    if name[0] == "A":
        df.at[index, 'brand'] = "Amd"
        name = name[4:]
        
    name = name.split("@")
    df.at[index, 'name'] = name[0]
    
df.head(5)

,name,price,Class,Socket,Clockspeed,Turbo Speed,Threads,Cores,Typical TDP,mt_score,st_score,brand
0,Ryzen Threadripper PRO 3995WX,nan,Desktop,sTRX4,2.7,4.2,128,64,280,88731,2665,Amd
1,Ryzen Threadripper 3990X,4999.99,Desktop,sTRX4,2.9,4.3,128,64,280,80558,2556,Amd
2,EPYC 7702,4955.00,Server,SP3,2.0,3.4,128,64,200,71859,2096,Amd
3,EPYC 7702P,4749.95,Server,SP3,2.0,3.4,128,64,200,68213,2210,Amd
4,EPYC 7742,6790.00,Server,SP3,2.3,3.4,128,64,225,67185,2376,Amd


The data looks mostly fine now but we have to do something about our missing values again dropping or replacing them, for Turbo speed we can just input the base clock speed. For price we may have to drop the row entirly or fine the price somewhere else as using the mean of the dataset can make unappropriate assumptions about cpu's.

In [8]:
for index, value in df.iterrows():
    clock = value['Clockspeed']
    
    if value['Turbo Speed'] == np.nan:
        df.at[index, 'Turbo Speed'] = clock
    
df.head(5)

,name,price,Class,Socket,Clockspeed,Turbo Speed,Threads,Cores,Typical TDP,mt_score,st_score,brand
0,Ryzen Threadripper PRO 3995WX,nan,Desktop,sTRX4,2.7,4.2,128,64,280,88731,2665,Amd
1,Ryzen Threadripper 3990X,4999.99,Desktop,sTRX4,2.9,4.3,128,64,280,80558,2556,Amd
2,EPYC 7702,4955.00,Server,SP3,2.0,3.4,128,64,200,71859,2096,Amd
3,EPYC 7702P,4749.95,Server,SP3,2.0,3.4,128,64,200,68213,2210,Amd
4,EPYC 7742,6790.00,Server,SP3,2.3,3.4,128,64,225,67185,2376,Amd


Now that our data looks good we need to make sure they are all the correct data type

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         740 non-null    object 
 1   price        740 non-null    object 
 2   Class        712 non-null    object 
 3   Socket       706 non-null    object 
 4   Clockspeed   740 non-null    float64
 5   Turbo Speed  436 non-null    float64
 6   Threads      740 non-null    int64  
 7   Cores        740 non-null    int64  
 8   Typical TDP  740 non-null    object 
 9   mt_score     740 non-null    int64  
 10  st_score     740 non-null    int64  
 11  brand        736 non-null    object 
dtypes: float64(2), int64(4), object(6)
memory usage: 69.5+ KB


In [10]:
cols = ['price']
df[cols] = df[cols].apply(pd.to_numeric, downcast='float', errors='coerce')
df.head(10)


,name,price,Class,Socket,Clockspeed,Turbo Speed,Threads,Cores,Typical TDP,mt_score,st_score,brand
0,Ryzen Threadripper PRO 3995WX,NaN,Desktop,sTRX4,2.7,4.2,128,64,280,88731,2665,Amd
1,Ryzen Threadripper 3990X,4999.990234,Desktop,sTRX4,2.9,4.3,128,64,280,80558,2556,Amd
2,EPYC 7702,4955.000000,Server,SP3,2.0,3.4,128,64,200,71859,2096,Amd
3,EPYC 7702P,4749.950195,Server,SP3,2.0,3.4,128,64,200,68213,2210,Amd
4,EPYC 7742,6790.000000,Server,SP3,2.3,3.4,128,64,225,67185,2376,Amd
5,Ryzen Threadripper 3970X,2061.830078,Desktop,sTRX4,3.7,4.5,64,32,280,64253,2713,Amd
6,Ryzen Threadripper PRO 3975WX,NaN,Desktop,sTRX4,3.5,4.2,64,32,280,63616,2668,Amd
7,Ryzen Threadripper 3960X,1349.989990,Desktop,sTRX4,3.8,4.5,48,24,280,55459,2703,Amd
8,EPYC 7502,2285.000000,Server,SP3,2.5,3.4,64,32,180,53591,2116,Amd
9,Ryzen 9 5950X,1679.989990,Desktop,AM4,3.4,4.9,32,16,105,46244,3529,Amd


Last thing to do is get rid of all the Null or NaN values

In [11]:
df['Turbo Speed'].fillna(df['Clockspeed'], inplace=True)
df.info()
df.dropna(subset=['Class', 'brand'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         740 non-null    object 
 1   price        497 non-null    float32
 2   Class        712 non-null    object 
 3   Socket       706 non-null    object 
 4   Clockspeed   740 non-null    float64
 5   Turbo Speed  740 non-null    float64
 6   Threads      740 non-null    int64  
 7   Cores        740 non-null    int64  
 8   Typical TDP  740 non-null    object 
 9   mt_score     740 non-null    int64  
 10  st_score     740 non-null    int64  
 11  brand        736 non-null    object 
dtypes: float32(1), float64(2), int64(4), object(5)
memory usage: 66.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 708 entries, 0 to 739
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         708 non-null    object 
 

Now that the data has been cleaned we can save it.

In [12]:
du.save_df(df, 'cpu_data_cleaned.csv')

Now we can implement pandas OneHotEncode on our class and brand

In [13]:
pd.get_dummies(df['Class'], prefix='class')
df = pd.concat([df,pd.get_dummies(df['Class'], prefix='class')], axis=1)
df = pd.concat([df,pd.get_dummies(df['brand'], prefix='brand')], axis=1)

In [14]:
df.head(60)

,name,price,Class,Socket,Clockspeed,Turbo Speed,Threads,Cores,Typical TDP,mt_score,st_score,brand,class_Desktop,class_Laptop,class_Server,brand_Amd,brand_Intel
0,Ryzen Threadripper PRO 3995WX,NaN,Desktop,sTRX4,2.7,4.2,128,64,280,88731,2665,Amd,1,0,0,1,0
1,Ryzen Threadripper 3990X,4999.990234,Desktop,sTRX4,2.9,4.3,128,64,280,80558,2556,Amd,1,0,0,1,0
2,EPYC 7702,4955.000000,Server,SP3,2.0,3.4,128,64,200,71859,2096,Amd,0,0,1,1,0
3,EPYC 7702P,4749.950195,Server,SP3,2.0,3.4,128,64,200,68213,2210,Amd,0,0,1,1,0
4,EPYC 7742,6790.000000,Server,SP3,2.3,3.4,128,64,225,67185,2376,Amd,0,0,1,1,0
5,Ryzen Threadripper 3970X,2061.830078,Desktop,sTRX4,3.7,4.5,64,32,280,64253,2713,Amd,1,0,0,1,0
6,Ryzen Threadripper PRO 3975WX,NaN,Desktop,sTRX4,3.5,4.2,64,32,280,63616,2668,Amd,1,0,0,1,0
7,Ryzen Threadripper 3960X,1349.989990,Desktop,sTRX4,3.8,4.5,48,24,280,55459,2703,Amd,1,0,0,1,0
8,EPYC 7502,2285.000000,Server,SP3,2.5,3.4,64,32,180,53591,2116,Amd,0,0,1,1,0
9,Ryzen 9 5950X,1679.989990,Desktop,AM4,3.4,4.9,32,16,105,46244,3529,Amd,1,0,0,1,0


Now that our class has been OneHot encoded we can drop our old collumns

In [15]:
df.drop(['Class'],axis=1,inplace=True)
df.drop(['brand'],axis=1,inplace=True)
df.head()

,name,price,Socket,Clockspeed,Turbo Speed,Threads,Cores,Typical TDP,mt_score,st_score,class_Desktop,class_Laptop,class_Server,brand_Amd,brand_Intel
0,Ryzen Threadripper PRO 3995WX,NaN,sTRX4,2.7,4.2,128,64,280,88731,2665,1,0,0,1,0
1,Ryzen Threadripper 3990X,4999.990234,sTRX4,2.9,4.3,128,64,280,80558,2556,1,0,0,1,0
2,EPYC 7702,4955.000000,SP3,2.0,3.4,128,64,200,71859,2096,0,0,1,1,0
3,EPYC 7702P,4749.950195,SP3,2.0,3.4,128,64,200,68213,2210,0,0,1,1,0
4,EPYC 7742,6790.000000,SP3,2.3,3.4,128,64,225,67185,2376,0,0,1,1,0


Saving the encoded data

In [16]:
du.save_df(df, 'cpu_data_encoded.csv')

Verify project structure

In [17]:
!tree ../

../
├── LICENSE
├── README.md
├── data
│   ├── Feature_importance.png
│   ├── cpu_data.csv
│   ├── cpu_data_cleaned.csv
│   └── cpu_data_encoded.csv
├── models
├── notebooks
│   ├── 0_setup_project_folders.ipynb
│   ├── 1_Retrieving_data.ipynb
│   ├── 2_Cleaning_data.ipynb
│   ├── 3_Simple_Graphs.ipynb
│   └── __pycache__
│       └── datautil.cpython-37.pyc
├── report
│   ├── interim.md
│   ├── proposal.md
│   └── report.md
├── src
│   ├── __pycache__
│   │   └── datautil.cpython-37.pyc
│   └── datautil.py
└── web
    └── atoris

9 directories, 16 files
